<a href="https://colab.research.google.com/github/ewertonUrso/LTV-CAC/blob/main/Sicredi_card_transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Coleta e organização dos dados

In [228]:
import pandas as pd

# 1. Agências Sicredi
f = '/content/drive/MyDrive/Sicredi/csv/db_entidade_agencia.csv'
agencias = pd.read_csv(f)

# 2. Transações em cartões de crédito dos clientes
f = '/content/drive/MyDrive/Sicredi/csv/db_cartoes_transacoes.csv'
cartoes_transacoes = pd.read_csv(f)

# 3. Adicionando as colunas dia, mês e ano da transação
def dia_mes(data):
  return data[0:2]

def mes(data):
  return data[3:5]

def ano(data):
  return data[6:10]

cartoes_transacoes['dia_transacao'] = cartoes_transacoes['dat_transacao'].apply(dia_mes)
cartoes_transacoes['mes_transacao'] = cartoes_transacoes['dat_transacao'].apply(mes)
cartoes_transacoes['ano_transacao'] = cartoes_transacoes['dat_transacao'].apply(ano)
cartoes_transacoes['ano_mes_transacao'] = cartoes_transacoes['ano_transacao'] + '-' + cartoes_transacoes['mes_transacao']
cartoes_transacoes['dat_transacao'] = cartoes_transacoes['ano_transacao'] + '-' + cartoes_transacoes['mes_transacao'] + '-' + cartoes_transacoes['dia_transacao']
cartoes_transacoes['dat_transacao'] = pd.to_datetime(cartoes_transacoes['dat_transacao'])

# 3. Removendo transações anteriores ao ano de 2019 e posteriores a 31/dez/2019
cartoes_transacoes = cartoes_transacoes[pd.to_datetime(cartoes_transacoes['dat_transacao']) > '2018-12-31']
cartoes_transacoes = cartoes_transacoes[pd.to_datetime(cartoes_transacoes['dat_transacao']) < '2020-01-01']
cartoes_transacoes = cartoes_transacoes.sort_values('dat_transacao', ascending = True).reset_index(drop = True)

# 4. Removendo transações com modalidade crédito ou débito não informada
cartoes_transacoes = cartoes_transacoes[cartoes_transacoes['nom_modalidade'] != '0']

# 5. Combinando os datasets de agência e de transações em cartão
d = cartoes_transacoes.merge(agencias, on = ['cod_cooperativa', 'cod_agencia'])

# 6. Selecionando variáveis
d = d[["num_cpf_cnpj",
       "cod_conta",
       "num_plastico",
       "cod_cooperativa",
       "des_nome_cooperativa",
       "cod_agencia",
       "des_nome_agencia",
       "nom_cidade_estabelecimento",
       "dat_transacao",
       "ano_mes_transacao",
       "vlr_transacao",
       "nom_modalidade"]]


2. Crescimento do TPV ao longo dos 24 meses entre jan/2019 e dez/2020

In [ ]:
dados = ['ano_mes_transacao', 'vlr_transacao']

tpv = d[dados].groupby('ano_mes_transacao', as_index = False).sum().sort_values('ano_mes_transacao', ascending = True).reset_index(drop = True)

tpv.head(12)


3. Quais as cooperativas com maior TPV?


In [ ]:
dados = ['des_nome_cooperativa', 'vlr_transacao']

tpv = d[dados].groupby('des_nome_cooperativa', as_index = False).sum().sort_values('vlr_transacao', ascending = False).reset_index(drop = True)

tpv.head(3)


4. Quais as agências com maior TPV?

In [ ]:
dados = ['des_nome_cooperativa', 'des_nome_agencia', 'vlr_transacao']
grupos = ['des_nome_cooperativa', 'des_nome_agencia']
ordenacao = ['des_nome_cooperativa', 'vlr_transacao']

tpv = d[dados].groupby(grupos, as_index = False).sum().sort_values(ordenacao, ascending = False).reset_index(drop = True)

tpv.head(13)


5. Crescimento do TPV das cooperativas
ao longo dos 24 meses entre jan/2019 e dez/2020

In [ ]:
dados = ['ano_mes_transacao', 'des_nome_cooperativa', 'vlr_transacao']
grupos = ['ano_mes_transacao', 'des_nome_cooperativa']
ordenacao = ['des_nome_cooperativa', 'ano_mes_transacao']

tpv = d[dados].groupby(grupos, as_index = False).sum()
tpv = tpv[['des_nome_cooperativa',
           'ano_mes_transacao',
           'vlr_transacao']].sort_values(ordenacao, ascending = True).reset_index(drop = True)

tpv.head(19)


6. Crescimento do TPV das agências ao longo dos 24 meses entre jan/2019 e dez/2020



In [ ]:
dados = ['ano_mes_transacao', 'des_nome_agencia', 'vlr_transacao']
grupos = ['ano_mes_transacao', 'des_nome_agencia']
ordenacao = ['des_nome_agencia', 'ano_mes_transacao']

tpv = d[dados].groupby(grupos, as_index = False).sum()
tpv = tpv[['des_nome_agencia',
           'ano_mes_transacao',
           'vlr_transacao']].sort_values(ordenacao, ascending = True).reset_index(drop = True)

tpv.head(94)


7. Proporção do TPV entre transações de Crédito e Débito

In [ ]:
dados = ['nom_modalidade', 'vlr_transacao']
grupos = ['nom_modalidade']
ordenacao = ['nom_modalidade', 'vlr_transacao']

tpv = d[dados].groupby(grupos, as_index = False).sum()

tpv.head(2)


8. Crescimento do TPV das modalidades Crédito e Débito


In [ ]:
dados = ['ano_mes_transacao', 'nom_modalidade', 'vlr_transacao']
grupos = ['ano_mes_transacao', 'nom_modalidade']
ordenacao = ['ano_mes_transacao', 'vlr_transacao']

tpv = d[dados].groupby(grupos, as_index = False).sum().sort_values(ordenacao, ascending = True).reset_index(drop = True)
tpv.head(24)


9. Clientes Ativos

In [ ]:
dados = ['ano_mes_transacao', 'num_cpf_cnpj']

ativos = d[dados].groupby('ano_mes_transacao', as_index = False).agg('nunique')
ativos.head(12)


10. Clientes Ativos nas diferentes cooperativas

In [ ]:
dados = ['des_nome_cooperativa', 'ano_mes_transacao', 'num_cpf_cnpj']
grupos = ['des_nome_cooperativa', 'ano_mes_transacao']
ordenacao = grupos

ativos = d[dados].groupby(grupos, as_index = False).agg('nunique').sort_values(ordenacao, ascending = True).reset_index(drop = True)
ativos.head(19)


11. Clientes Ativos nas diferentes agências

In [ ]:
dados = ['des_nome_cooperativa', 'des_nome_agencia', 'ano_mes_transacao', 'num_cpf_cnpj']
grupos = ['des_nome_cooperativa', 'des_nome_agencia', 'ano_mes_transacao']
ordenacao = grupos

ativos = d[dados].groupby(grupos, as_index = False).agg('nunique').sort_values(ordenacao, ascending = True).reset_index(drop = True)
ativos.head(94)
